In [ ]:
!nvidia-smi

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
!git clone https://github.com/gangweiX/IGEV.git

In `/content/IGEV/IGEV-Stereo/demo_imgs.py`, change
```
from igev_stereo import IGEVStereo
from utils.utils import InputPadder
```
to
```
from core.igev_stereo import IGEVStereo
from core.utils.utils import InputPadder
```

Also change line 52 to
```
plt.imsave(output_directory / f"{file_stem}.png", disp.squeeze(), cmap='gray')

```
to avoid color-mapping the disparity map.

In [ ]:
!conda install pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch -c nvidia
!pip install opencv-python
!pip install scikit-image
!pip install tensorboard
!pip install matplotlib
!pip install tqdm
!pip install timm==0.5.4

# Generate left disparity map

In [ ]:
!python IGEV/IGEV-Stereo/demo_imgs.py --restore_ckpt /content/sceneflow.pth -l IGEV/roses_L.png -r IGEV/roses_R.png

# Generate right disparity map

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

'''
Rotate left and right input images
'''
left_img = cv2.imread("/content/IGEV/roses_L.png")
rotated_left_img = left_img[:, ::-1]
# cv2_imshow(rotated_left_img)
cv2.imwrite("/content/IGEV/roses_L_rotated.png", rotated_left_img)

right_img = cv2.imread("/content/IGEV/roses_R.png")
rotated_right_img = right_img[:, ::-1]
# cv2_imshow(rotated_right_img)
cv2.imwrite("/content/IGEV/roses_R_rotated.png", rotated_right_img)


'''
Generate rotated version of right disparity map
'''
!python IGEV/IGEV-Stereo/demo_imgs.py --restore_ckpt ./sceneflow.pth -l /content/IGEV/roses_R_rotated.png -r /content/IGEV/roses_L_rotated.png
'''
Correct orientation of right disparity map
'''
right_disp_rotated = cv2.imread("/content/demo-output/IGEV.png")
# cv2_imshow(right_disp_rotated)
right_disp = right_disp_rotated[:, ::-1]
cv2_imshow(right_disp)
cv2.imwrite("/content/demo-output/roses_right_disparity.png", right_disp)

# Generate report

In [ ]:
!unzip test.zip

In [ ]:
import cv2
from google.colab import files

imgs = [
    "art",
    "flowerbed",
    "lilies",
    "statue",
    "sunflowers",
    "tree",
    "trees",
    "woods",
    "roses",
    "rock",
    "peaches",
    "mug",
    "grapes"
]

for img in imgs:
  !python IGEV/IGEV-Stereo/demo_imgs.py --restore_ckpt /content/eth3d.pth -l /content/test/{img}/{img}_L.png -r /content/test/{img}/{img}_R.png --output_directory /content/test/{img}
  !mv /content/test/{img}/{img}.png /content/test/{img}/left_disparity.png

  left_img = cv2.imread(f"/content/test/{img}/{img}_L.png")
  rotated_left_img = left_img[:, ::-1]
  cv2.imwrite(f"/content/test/{img}/{img}_L_rotated.png", rotated_left_img)
  right_img = cv2.imread(f"/content/test/{img}/{img}_R.png")
  rotated_right_img = right_img[:, ::-1]
  cv2.imwrite(f"/content/test/{img}/{img}_R_rotated.png", rotated_right_img)

  !python IGEV/IGEV-Stereo/demo_imgs.py --restore_ckpt /content/eth3d.pth -l /content/test/{img}/{img}_R_rotated.png -r /content/test/{img}/{img}_L_rotated.png --output_directory /content/test/{img}
  right_disp_rotated = cv2.imread(f"/content/test/{img}/{img}.png")
  right_disp = right_disp_rotated[:, ::-1]
  cv2.imwrite(f"/content/test/{img}/right_disparity.png", right_disp)
  !rm -f /content/test/{img}/{img}_R_rotated.png
  !rm -f /content/test/{img}/{img}_L_rotated.png
  !rm -f /content/test/{img}/{img}.png

!zip -r test.zip test
files.download(f"test.zip")
